In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from data_tools import *
from data_tools import IOTools as it
from data_tools import PandasTools as pt
from data_tools import ChartTools as ct
ct.IPYTHON_MODE = True

ImportError: DLL load failed: 找不到指定的程序。

 #### 例0：词典创建、语句的One-hot编码

In [7]:
from keras.preprocessing.text import Tokenizer
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

# 初始化1000个词的词典
tokenizer = Tokenizer(num_words=1000)
# 根据样本建立词典（词+索引）
tokenizer.fit_on_texts(samples)

# 返回词索引序列，如 [[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
sequences = tokenizer.texts_to_sequences(samples)

# 返回每一句在1000维向量中，所有词所在维度位置（1或0）,相当于句子的One-hot
one_hot_results = tokenizer.texts_to_matrix(samples, mode='binary')

# 返回词典如：{'the': 1, 'cat': 2, 'sat': 3, 'on': 4, 'mat': 5, 'dog': 6, 'ate': 7, 'my': 8, 'homework': 9}
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9 unique tokens.


 #### 例1：词向量训练
> 词的embeddings（相对于one-hot，低维度，稠密），embedding方法：GloVe（基于分解词共现矩阵）、Word2Vector、keras embedding等


In [2]:
from keras.layers import Embedding
from keras.datasets import imdb
from keras import preprocessing

# 只考虑出现频率前10000词汇
max_features = 10000
# 最大句长
maxlen = 20
# 载入数据，数据集为词典索引序列
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
# 截取或填充前20个词汇
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

Using TensorFlow backend.


In [19]:
x_train.shape

(25000, 20)

In [4]:
from keras.models import Sequential
from keras.layers import Flatten, Dense
model = Sequential()
# 建立10000个词的Embedding，词维度为8，输入句长为20（经过填充或裁减）
model.add(Embedding(10000, 8, input_length=maxlen))
# Flatten，拼接句子的词向量，每句由160维表示
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


In [5]:
# 此时Embedding层参与训练，导致预测效果不好
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 8s 386us/step - loss: 0.6759 - acc: 0.6050 - val_loss: 0.6398 - val_acc: 0.6814
Epoch 2/10
20000/20000 [==============================] - 2s 116us/step - loss: 0.5657 - acc: 0.7427 - val_loss: 0.5467 - val_acc: 0.7206
Epoch 3/10
20000/20000 [==============================] - 2s 116us/step - loss: 0.4752 - acc: 0.7808 - val_loss: 0.5113 - val_acc: 0.7384
Epoch 4/10
20000/20000 [==============================] - 2s 116us/step - loss: 0.4263 - acc: 0.8077 - val_loss: 0.5008 - val_acc: 0.7452
Epoch 5/10
20000/20000 [==============================] - 2s 118us/step - loss: 0.3930 - acc: 0.8258 - val_loss: 0.4981 - val_acc: 0.7538
Epoch 6/10
20000/20000 [==============================] - 2s 122us/step - loss: 0.3668 - acc: 0.8395 - val_loss: 0.5014 - val_acc: 0.7530
Epoch 7/10
20000/20000 [==============================] - 2s 114us/step - loss: 0.3435 - acc: 0.8533 - val_loss: 0.5052 - v

#### 例2：利用已训练词向量+imdb情感分类
1. 下载imdb评论文本源数据，解压
2. 对评论文本数据建立词典
3. 根据建立的词典，抽取词向量，并建立以词典索引为行号的词向量数据
4. embedding层载入词向量数据
5. 建立训练网络，做模型训练

结论：
1. 预训练的词向量有助于对小数据预测结果有额外提升
2. 建议训练时不锁定embedding层，这样可以在训练文本环境中修正词向量

In [37]:
import os
# 下载以下imdb源文本数据并解压 做情感分类
# http://ai.stanford.edu/~amaas/data/sentiment/
imdb_dir = '.\\aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
# 获取文本list以及对应情感list
def get_raw_data (data_path):
    labels = []
    texts = []
    for label_type in ['neg', 'pos']:
        dir_name = os.path.join(train_dir, label_type)
        for fname in os.listdir(dir_name):
            if fname[-4:] == '.txt':            
                try:
                    f = open(os.path.join(dir_name, fname))
                    texts.append(f.read())
                    if label_type == 'neg':
                        labels.append(0)
                    else:
                        labels.append(1)
                except :
                    print("Error: 读取文件失败 %s"%fname)
                finally:
                    f.close()
    return texts,labels
texts,labels=get_raw_data(train_dir)

Error: 读取文件失败 1405_3.txt
Error: 读取文件失败 3832_4.txt
Error: 读取文件失败 4526_4.txt
Error: 读取文件失败 4559_1.txt
Error: 读取文件失败 4814_3.txt
Error: 读取文件失败 5241_3.txt
Error: 读取文件失败 6122_2.txt
Error: 读取文件失败 6929_1.txt
Error: 读取文件失败 9435_2.txt
Error: 读取文件失败 10327_7.txt
Error: 读取文件失败 11505_7.txt
Error: 读取文件失败 11668_7.txt
Error: 读取文件失败 12467_7.txt
Error: 读取文件失败 1990_10.txt
Error: 读取文件失败 2538_10.txt
Error: 读取文件失败 4942_7.txt
Error: 读取文件失败 9407_8.txt


In [ ]:
# 最大句长
maxlen = 100
training_samples = 200
validation_samples = 10000
# 只考虑出现频率前10000词汇
max_words = 10000

# 建立词典
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
# 打印词典大小
print('Found %s unique tokens.' % len(word_index))

# 填充或切句，并转化数据格式为numpy
data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# 建立训练集，交叉验证集
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

In [ ]:
# 下载并载入glove Word embedding (nlp.stanford.edu/projects/glove/)，建立词向量词典
# 文件格式（词+向量）： word 1 2 3 4 
glove_dir = '/Users/fchollet/Downloads/glove.6B'
# 词向量词典
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
# 根据imdb建立的词典，抽取词向量，并建立以词典索引为行号的词向量数据
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
# Embedding层载入词典索引为行号的词向量
model.layers[0].set_weights([embedding_matrix])
# Embedding层不参与训练
model.layers[0].trainable = False
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
# 词索引序列、情感label训练
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights('pre_trained_glove_model.h5')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# 预测
test_dir = os.path.join(imdb_dir, 'test')
texts,labels=get_raw_data(test_dir)

sequences = tokenizer.texts_to_sequences(texts)
x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels)

model.load_weights('pre_trained_glove_model.h5')
model.evaluate(x_test, y_test)